In [270]:
from typing import Optional
from datetime import date
import math
import pickle
from dataclasses import asdict, dataclass

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [275]:
df = pd.read_excel('1.ОСНОВНОЙ Факт объемы_м3_2017_2022_по дням1.xlsx', header=1, sheet_name=5)
df

,Unnamed: 0,МСК 107,СЗ,НН,Урал,Всего Хранение,Unnamed: 6,Unnamed: 7,МСК 107.1,Центр Дем,...,НН.3,Урал.3,Unnamed: 24,МСК,СЗ.4,Урал.4,Unnamed: 28,МСК.1,СЗ.5,Урал.5
0,2022-01-01,9654.64,6980.78,3971.04,4074.89,24681.35,NaN,NaN,0.00,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-01-02,9654.64,6980.78,3971.04,4074.89,24681.35,NaN,NaN,0.00,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-01-03,9654.69,6980.78,3971.04,4074.89,24681.40,NaN,NaN,0.04,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-01-04,9654.69,6980.78,3971.04,4074.89,24681.40,NaN,NaN,0.00,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-01-05,9654.69,6980.78,3971.04,4074.89,24681.40,NaN,NaN,0.00,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,2022-12-28,0.00,0.00,0.00,0.00,0.00,NaN,NaN,0.00,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
362,2022-12-29,0.00,0.00,0.00,0.00,0.00,NaN,NaN,0.00,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
363,2022-12-30,0.00,0.00,0.00,0.00,0.00,NaN,NaN,0.00,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
364,2022-12-31,0.00,0.00,0.00,0.00,0.00,NaN,NaN,0.00,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [280]:
@dataclass
class Row:
    date: date
    keeping_actual: float
    keeping_reserved: Optional[float]
    processing_actual: float
    receiving_actual: float
    shipment_actual: float  
    processing_reserved: Optional[float]
        
month_ru_to_num = {"январь": 1, "февраль": 2, "март": 3, "апрель": 4, "май": 5, "июнь": 6, "июль": 7, "август": 8, "сентябрь": 9, "октябрь": 10, "ноябрь": 11, "декабрь": 12}

In [281]:
def fillna(num: float):
    if isinstance(num, str):
        num = num.replace(',', '.')
        return float(num)

    return num if not pd.isnull(num) else 0

def process_sheet(warehouse_name: str, month_column: str, date_column: str, reserved: bool = True):
    results = []
    if not reserved:
        for idx, row in df.iterrows():
            cur_date = row[date_column]
            row_ = Row(date=cur_date, 
                       keeping_actual=fillna(row[warehouse_name]), 
                       processing_actual=fillna(row[f'{warehouse_name}.1']),
                       receiving_actual=fillna(row[f'{warehouse_name}.2']),
                       shipment_actual=fillna(row[f'{warehouse_name}.3']),
                       keeping_reserved=None, 
                       processing_reserved=None)
            results.append(row_)
        return results
            
        
    processing_reserved_month = {}
    results = []
    for idx, row in df.iterrows():
        if idx == 12:
            break

        month = month_ru_to_num[row[month_column].lower()]
        try:
            processing_reserved_month[month] = row[f'{warehouse_name}.5']
        except KeyError:
            processing_reserved_month[month] = row['МСК.1']
    
    for idx, row in df.iterrows():
        cur_date = row[date_column]
        try:
            row_ = Row(date=cur_date, 
                       keeping_actual=fillna(row[warehouse_name]), 
                       processing_actual=fillna(row[f'{warehouse_name}.1']),
                       receiving_actual=fillna(row[f'{warehouse_name}.2']),
                       shipment_actual=fillna(row[f'{warehouse_name}.3']),
                       keeping_reserved=fillna(row[f'{warehouse_name}.4']), 
                       processing_reserved=processing_reserved_month[cur_date.month])
        except KeyError:
            row_ = Row(date=cur_date, 
                       keeping_actual=fillna(row[warehouse_name]), 
                       processing_actual=fillna(row[f'{warehouse_name}.1']),
                       receiving_actual=fillna(row[f'{warehouse_name}.2']),
                       shipment_actual=fillna(row[f'{warehouse_name}.3']),
                       keeping_reserved=fillna(row['МСК']), 
                       processing_reserved=processing_reserved_month[cur_date.month])
        results.append(row_)
    return results

warehouses = {"МСК": [], "СЗ": [], "Урал": [], "МСК 101": [], "МСК 107": [], "МСК Демонтаж": [], "НН": []}    
# ## SHEET 1
# df = pd.read_excel('1.ОСНОВНОЙ Факт объемы_м3_2017_2022_по дням1.xlsx', header=1, sheet_name=1)
# for key, value in warehouses.items():
#     if key in ['МСК', 'СЗ', 'Урал']:
#         value.extend(process_sheet(key, 'Unnamed: 26', 'Unnamed: 0'))

# ### SHEET 2
# df = pd.read_excel('1.ОСНОВНОЙ Факт объемы_м3_2017_2022_по дням1.xlsx', header=1, sheet_name=2)
# for key, value in warehouses.items():
#     if key in ['МСК', 'СЗ', 'Урал']:
#         value.extend(process_sheet(key, 'Unnamed: 24', 'склад'))

# ### SHEET 3
# df = pd.read_excel('1.ОСНОВНОЙ Факт объемы_м3_2017_2022_по дням1.xlsx', header=1, sheet_name=3, skipfooter=14)
# for key, value in warehouses.items():
#     if key in ['МСК', 'СЗ', 'Урал']:
#         value.extend(process_sheet(key, 'Unnamed: 25', 'Unnamed: 0'))
        
# ## SHEET 4
# df = pd.read_excel('1.ОСНОВНОЙ Факт объемы_м3_2017_2022_по дням1.xlsx', header=1, sheet_name=4)
# df = df.drop(df.index[59])
# df = df.drop(df.index[365:])
# for key, value in warehouses.items():
#     if key in ['МСК 101', 'МСК 107', 'СЗ', 'МСК Демонтаж', 'Урал']:
#         value.extend(process_sheet(key, 'Unnamed: 33', 'дата'))
        
## SHEET 5
df = pd.read_excel('1.ОСНОВНОЙ Факт объемы_м3_2017_2022_по дням1.xlsx', header=1, sheet_name=5)
# df = df.drop(df.index[59])
# df = df.drop(df.index[365:])
for key, value in warehouses.items():
    if key in ['МСК 107', 'СЗ', 'НН', 'Урал']:
        value.extend(process_sheet(key, 'Unnamed: 28', 'Unnamed: 0', reserved = False))
    

dfs = {key: (pd.DataFrame((asdict(row) for row in warehouse)).fillna(0.0)) for key, warehouse in warehouses.items()}
dfs
with open('dfs.pkl', 'wb') as f:
    pickle.dump(dfs, f)
    
dfs

{'МСК': Empty DataFrame
 Columns: []
 Index: [],
 'СЗ':                     date  keeping_actual  keeping_reserved  processing_actual  \
 0    2022-01-01 00:00:00         6980.78               0.0                0.0   
 1    2022-01-02 00:00:00         6980.78               0.0                0.0   
 2    2022-01-03 00:00:00         6980.78               0.0                0.0   
 3    2022-01-04 00:00:00         6980.78               0.0                0.0   
 4    2022-01-05 00:00:00         6980.78               0.0                0.0   
 ..                   ...             ...               ...                ...   
 361  2022-12-28 00:00:00            0.00               0.0                0.0   
 362  2022-12-29 00:00:00            0.00               0.0                0.0   
 363  2022-12-30 00:00:00            0.00               0.0                0.0   
 364  2022-12-31 00:00:00            0.00               0.0                0.0   
 365                  0.0            0.00  